# Évaluation des intéractions entre le véhicule et son environnement

In [1]:
%matplotlib inline
from nuscenes.nuscenes import NuScenes
nusc = NuScenes(version='v1.0-mini', dataroot='/Users/tiffanydarini/nuscenes-devkit/python-sdk/tutorials/data/sets/nuscenes/v1.0-mini', verbose=True)

Loading NuScenes tables for version v1.0-mini...
23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
Done loading in 0.351 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.


### Étape 1 : Récupération des données sur le véhicule ego

In [2]:
# Fonction qui renvoie les tables ego_pose d'une scène donnée
def get_poses(scene_token):
    pose_list = []
    scene_rec = nusc.get('scene', scene_token)
    sample_rec = nusc.get('sample', scene_rec['first_sample_token'])
    sd_rec = nusc.get('sample_data', sample_rec['data']['LIDAR_TOP'])
    
    ego_pose = nusc.get('ego_pose', sd_rec['token'])
    pose_list.append(ego_pose)

    while sd_rec['next'] != '':
        sd_rec = nusc.get('sample_data', sd_rec['next'])
        ego_pose = nusc.get('ego_pose', sd_rec['token'])
        pose_list.append(ego_pose)

    return pose_list

In [3]:
# On utilise ce qu'on a déjà (liste ego_poses)
scene_name = 'scene-0061'
scene_token = nusc.field2token('scene','name',scene_name)[0]
Liste_ego_poses = get_poses(scene_token) 

# Création des liste de coordonnées du véhicule égo 
X_coord =[]
Y_coord =[]
Z_coord =[]
T_timestamp = [] # liste des timestamps
for k in range(0,len(Liste_ego_poses)):
    X_coord.append(Liste_ego_poses[k]['translation'][0])
    Y_coord.append(Liste_ego_poses[k]['translation'][1])
    Z_coord.append(Liste_ego_poses[k]['translation'][2])
    T_timestamp.append(Liste_ego_poses[k]['timestamp'])

### Étape 2 : On récupère les échantillons de la scène 

In [4]:
# Ensemble des token des échantillons de la scene 
sample_token = nusc.field2token('sample','scene_token',scene_token)
len(sample_token)

39

### Étape 3 : On récupère les sample_annotation de la scène (boites d'objets dans la scène)

In [5]:
# Ensemble des sample_annotation de la scène :

sample_annotation_token = []
for k in sample_token : 
    L = nusc.field2token('sample_annotation','sample_token',k)
    sample_annotation_token = sample_annotation_token + L

### Étape 4 : Liste des instances dans la scène

In [6]:
# On créé la liste des token des instances présents dans les sample_annotation
L_instance_token = []
for k in sample_annotation_token :
    exemple = nusc.get('sample_annotation', k)
    L_instance_token.append(exemple['instance_token'])

In [7]:
# Puisque pour un même objet, on peut avoir plusieurs annotations, on supprime les doublons de L
L_instance_token = list(set(L_instance_token))
len(L_instance_token)
print('Il y a '+str(len(L_instance_token))+' instances dans la scène ' +scene_name)

Il y a 227 instances dans la scène scene-0061


### Étape 5 : Séparation de chaque catégorie 

In [8]:
# Liste des catégories d'objet dans la scène 
L_category_name = []
for k in sample_annotation_token :
    exemple = nusc.get('sample_annotation', k)
    L_category_name.append(exemple['category_name'])
L_category_name = list(set(L_category_name))
L_category_name

['human.pedestrian.construction_worker',
 'movable_object.pushable_pullable',
 'vehicle.motorcycle',
 'vehicle.bus.rigid',
 'movable_object.trafficcone',
 'movable_object.barrier',
 'human.pedestrian.adult',
 'vehicle.car',
 'movable_object.debris',
 'vehicle.construction',
 'vehicle.truck',
 'vehicle.bicycle']

In [9]:
# Pour l'instant, travail à la main pour chaque catégorie trouvée
k = 10
catego_name = L_category_name[k]
catego_token = nusc.field2token('category','name',catego_name)
category = nusc.get('category',catego_token[0])
category

{'token': '6021b5187b924d64be64a702e5570edf',
 'name': 'vehicle.truck',
 'description': 'Vehicles primarily designed to haul cargo including pick-ups, lorrys, trucks and semi-tractors. Trailers hauled after a semi-tractor should be labeled as vehicle.trailer'}

In [10]:
# Ensemble des instances appartenant à la fois à la scène et à la catégorie
L_instance_category = nusc.field2token('instance','category_token',catego_token[0]) # instance dans la catégorie
L_instance_category = [x for x in L_instance_category if x in L_instance_token]# et dans la scène 

In [11]:
L=[]
for k in L_instance_category : 
    instance_category = nusc.get('instance',k)
    nbr_annotations = instance_category['nbr_annotations']
    if nbr_annotations == 1:
        L.append([instance_category['first_annotation_token']])
    else : 
        H = []
        first_token = instance_category['first_annotation_token']
        last_token = instance_category['last_annotation_token']
        current_token = first_token 
        
        i = 0 
        while i!= nbr_annotations :
            current_ann = nusc.get('sample_annotation', current_token)
            H.append(current_ann['token'])
            current_token = current_ann['next'] #On passe au suivant
            i+=1
        L.append(H)
# Il peut y avoir plusieurs éléments de la catégorie dans la scène, ici chaque ligne de L représente 
# les token des sample_annotation pour un élément de cette catégorie. 

In [12]:
# Maintenant qu'on a les token de tous les sample_annotation pour tous les éléments d'une même catégorie, 
# on va pouvoir récupérer leurs positions et tracer leurs trajectoires

X = []
Y = []
Z = []
T_time = []
for k in L :
    x = []
    y = []
    z = []
    t = []
    for j in k :
        sample_annotation = nusc.get('sample_annotation',j)
        x.append(sample_annotation['translation'][0])
        y.append(sample_annotation['translation'][1])
        z.append(sample_annotation['translation'][2])
        t.append(nusc.get('sample',sample_annotation['sample_token'])['timestamp'])
    X.append(x)
    Y.append(y)
    Z.append(z)
    T_time.append(t)

### Étape 6 : Fonction pour savoir si un objet est immobile ou pas 

In [13]:
# On crée une fonction qui renvoie la liste des rangs dans les listes X,Y des objets immobiles de 
# cette liste et donc d'une même catégorie, ainsi que la liste des objets mobiles 
from math import sqrt

def immobile(X,Y):
    Immo = []
    Mobile = []
    n = len(X) # Nombre d'élément dans la catégorie 
    for i in range(0,n-1):
        X_coord = X[i]
        Y_coord = Y[i]
        if len(X_coord)==1 :
            Immo.append(i)
        else : 
            d = 0 
            for k in range(0,len(X_coord)-1):
                d+= sqrt((X_coord[k+1]-X_coord[k])**2+(Y_coord[k+1]-Y_coord[k])**2)
            if d<2 : # Si la distance totale parcourue par l'objet est inférieur à 2m, on considère que l'objet bouge pas
                Immo.append(i)
            else :
                Mobile.append(i)
    return Immo,Mobile  

In [14]:
Immo, Mobile = immobile(X,Y)
len(Mobile)

1

### Étape 7 : On garde uniquement les objets mobiles

In [15]:
X_mobile=[X[i] for i in Mobile]
Y_mobile=[Y[i] for i in Mobile]
Z_mobile=[Z[i] for i in Mobile]
T_mobile = [T_time[i] for i in Mobile]
len(X_mobile)

1

### Étape 8 : Création des vecteurs

In [16]:
import numpy as np
import datetime 
from datetime import timedelta

In [17]:
# On commence par le véhicule ego 
V_ego = []
for k in range(0,len(X_coord)-1):
    t2 = datetime.datetime.fromtimestamp(T_timestamp[k+1]/ 1e6)
    t1 = datetime.datetime.fromtimestamp(T_timestamp[k]/ 1e6)
    t = (t2-t1).total_seconds()
    v = np.array([X_coord[k+1]-X_coord[k],Y_coord[k+1]-Y_coord[k]])/t # Le vecteur a pour norme la vitesse à l'instant k
    V_ego.append(v)

In [18]:
# On va travailler avec une df
import pandas as pd

In [19]:
df_vecteurs = pd.DataFrame({'V_ego':V_ego})
df_vecteurs

,V_ego
0,"[-3.1712023320455462, -8.567313420091782]"
1,"[-3.1207235773484103, -8.562076268008871]"
2,"[-3.1653930094341445, -8.53617857224011]"
3,"[-3.1740825254365994, -8.49344994582648]"
4,"[-3.243236261012891, -8.504405724567667]"
...,...
376,"[1.9723523754827463, -0.4720986514141277]"
377,"[1.783906470998763, -0.4205889639945817]"
378,"[1.8364963045822984, -0.414838869174028]"
379,"[1.9999102240345528, -0.44535267086528335]"


In [20]:
# On passe aux éléments de la catégorie 
i = 0 
for j in range(0,len(X_mobile)) :
    i = i+ 1
    V = []
    for k in range(0,len(X_mobile[j])-1):
        t2 = datetime.datetime.fromtimestamp(T_mobile[j][k+1]/ 1e6)
        t1 = datetime.datetime.fromtimestamp(T_mobile[j][k]/ 1e6)
        t = (t2-t1).total_seconds()
        v = np.array([X_mobile[j][k+1]-X_mobile[j][k],Y_mobile[j][k+1]-Y_mobile[j][k]])/t
        V.append(v)
    # On a pas autant de vecteur pour un objet que pour le véhicule ego
    nbr_case_vide = len(V_ego)-len(V)
    Complement = [np.nan]*nbr_case_vide # on va donc compléter par des cases vides pour mettre dans la df
    Vf = V + Complement
    df_vecteurs[catego_name+ str(i)]=Vf

In [21]:
df_vecteurs

,V_ego,vehicle.truck1
0,"[-3.1712023320455462, -8.567313420091782]","[-1.396290428409181, -2.914606238097739]"
1,"[-3.1207235773484103, -8.562076268008871]","[-1.6448534262932504, -3.462371024407394]"
2,"[-3.1653930094341445, -8.53617857224011]","[-1.564174202140988, -4.147765393897798]"
3,"[-3.1740825254365994, -8.49344994582648]","[-2.018207831445658, -3.9684759933176865]"
4,"[-3.243236261012891, -8.504405724567667]","[-2.387841795586698, -4.164431271492787]"
...,...,...
376,"[1.9723523754827463, -0.4720986514141277]",NaN
377,"[1.783906470998763, -0.4205889639945817]",NaN
378,"[1.8364963045822984, -0.414838869174028]",NaN
379,"[1.9999102240345528, -0.44535267086528335]",NaN


In [22]:
# On crée une nouvelle dataframe pour insérer les produits scalaires entre chaque vecteur 
L =[('v'+str(i)+' . v_objet') for i in range(0,len(V_ego))]
df_ps = pd.DataFrame(index=L)
df_ps

""
v0 . v_objet
v1 . v_objet
v2 . v_objet
v3 . v_objet
v4 . v_objet
...
v376 . v_objet
v377 . v_objet
v378 . v_objet
v379 . v_objet


In [34]:
for i in range(1,len(df_vecteurs.columns)):
    nom_col = df_vecteurs.columns[i] # c'est un str 
    liste_vecteur = df_vecteurs[nom_col].values
    newlist = [x for x in liste_vecteur if np.any(np.isnan(x) == False)]
    L =[]
    for j in range(0,len(df_vecteurs)):
        l=[]
        for k in range(0,len(newlist)):
            l.append(np.dot(df_vecteurs['V_ego'].values[j],newlist[k]))
        L.append(l)
    df_ps[nom_col] = L

In [35]:
df_ps

,vehicle.truck1
v0 . v_objet,"[29.39826460072205, 34.87938076407666, 40.4955..."
v1 . v_objet,"[29.312517362569707, 34.778217647835774, 40.39..."
v2 . v_objet,"[29.299407277393502, 34.76202488482389, 40.357..."
v3 . v_objet,"[29.187003244324725, 34.628375506986075, 40.19..."
v4 . v_objet,"[29.31549372446092, 34.78005623675302, 40.3472..."
...,...
v376 . v_objet,"[-1.377995068927523, -1.6096498713523442, -1.1..."
v377 . v_objet,"[-1.2650003124991003, -1.4780296288886523, -1...."
v378 . v_objet,"[-1.3551902559970614, -1.584441158541002, -1.1..."
v379 . v_objet,"[-1.494427830839663, -1.7475830010357358, -1.2..."


### Étape 9 : Automatisation du calcul du produit scalaire pour tous les éléments

In [25]:
#imports nécessaires
import matplotlib.pyplot as plt
from math import sqrt
import datetime 
from datetime import timedelta
import pandas as pd
import numpy as np 

def produit_scalaire(scene_name):
    
    # On crée une dataframe vide 
    df = pd.DataFrame()
    
    scene_token = nusc.field2token('scene','name',scene_name)[0]
    sample_token = nusc.field2token('sample','scene_token',scene_token)# Ensemble des token des échantillons de la scene 
    
    # Pour tracer la trajectoire du véhicule ego
    Liste_ego_poses = get_poses(scene_token) 
    X_coord =[]
    Y_coord =[]
    Z_coord =[]
    T_timestamp = [] # liste des timestamps
    for k in range(0,len(Liste_ego_poses)):
        X_coord.append(Liste_ego_poses[k]['translation'][0])
        Y_coord.append(Liste_ego_poses[k]['translation'][1])
        Z_coord.append(Liste_ego_poses[k]['translation'][2])
        T_timestamp.append(Liste_ego_poses[k]['timestamp'])

    # Ensemble des sample_annotation de la scène :
    sample_annotation_token = []
    for k in sample_token : 
        L = nusc.field2token('sample_annotation','sample_token',k)
        sample_annotation_token = sample_annotation_token + L
    
    # On créé la liste des token des instances présents dans les sample_annotation
    L_instance_token = []
    for k in sample_annotation_token :
        exemple = nusc.get('sample_annotation', k)
        L_instance_token.append(exemple['instance_token'])
    # Puisque pour un même objet, on peut avoir plusieurs annotations, on supprime les doublons de L
    L_instance_token = list(set(L_instance_token))
    
    # Liste des catégories d'objet dans la scène 
    L_category_name = []
    for k in sample_annotation_token :
        exemple = nusc.get('sample_annotation', k)
        L_category_name.append(exemple['category_name'])
    L_category_name = list(set(L_category_name)) #on supprime les doublons 
   
    # On commence par le véhicule ego 
    V_ego = []
    for k in range(0,len(X_coord)-1):
        t2 = datetime.datetime.fromtimestamp(T_timestamp[k+1]/ 1e6)
        t1 = datetime.datetime.fromtimestamp(T_timestamp[k]/ 1e6)
        t = (t2-t1).total_seconds()
        v = np.array([X_coord[k+1]-X_coord[k],Y_coord[k+1]-Y_coord[k]])/t # Le vecteur a pour norme la vitesse à l'instant k
        V_ego.append(v)
            
    df_vecteurs = pd.DataFrame({'V_ego':V_ego})
    
    # On crée une nouvelle dataframe pour insérer les produits scalaires entre chaque vecteur 
    L =[('v'+str(i)+' . v_objet') for i in range(0,len(V_ego))]
    df_ps = pd.DataFrame(index=L)
    
    # Boucle sur chaque catégorie trouvée
    for j in range(0,len(L_category_name)):   
        catego_name = L_category_name[j]
        catego_token = nusc.field2token('category','name',catego_name)
        category = nusc.get('category',catego_token[0])

        # Ensemble des instances appartenant à la fois à la scène et à la catégorie
        L_instance_category = nusc.field2token('instance','category_token',catego_token[0]) # instance dans la catégorie
        L_instance_category = [x for x in L_instance_category if x in L_instance_token]# et dans la scène 
        L_instance_category
        
        # Il peut y avoir plusieurs éléments de la catégorie dans la scène, ici chaque ligne de L représente 
        # les token des sample_annotation pour un élément de cette catégorie. 

        L=[]
        for k in L_instance_category : 
            instance_category = nusc.get('instance',k)
            nbr_annotations = instance_category['nbr_annotations']
            if nbr_annotations == 1:
                L.append([instance_category['first_annotation_token']])
            else : 
                H = []
                first_token = instance_category['first_annotation_token']
                last_token = instance_category['last_annotation_token']
                current_token = first_token 
        
                s = 0 
                while s!= nbr_annotations :
                    current_ann = nusc.get('sample_annotation', current_token)
                    H.append(current_ann['token'])
                    current_token = current_ann['next'] #On passe au suivant
                    s+=1
                L.append(H)
        
        # Maintenant qu'on a les token de tous les sample_annotation pour tous les éléments d'une même catégorie, 
        # on va pouvoir récupérer leurs positions et tracer leurs trajectoires

        X = []
        Y = []
        Z = []
        T_time = []
        for k in L :
            x = []
            y = []
            z = []
            t=[]
            for j in k :
                sample_annotation = nusc.get('sample_annotation',j)
                x.append(sample_annotation['translation'][0])
                y.append(sample_annotation['translation'][1])
                z.append(sample_annotation['translation'][2])
                t.append(nusc.get('sample',sample_annotation['sample_token'])['timestamp'])
            X.append(x)
            Y.append(y)
            Z.append(z)
            T_time.append(t)
    
        # On crée les listes des objets immobiles et mobiles de la catégorie
        Immo, Mobile = immobile(X,Y)
        
        # On conserve uniquement les objets mobiles 
        X_mobile=[X[i] for i in Mobile]
        Y_mobile=[Y[i] for i in Mobile]
        Z_mobile=[Z[i] for i in Mobile]
        T_mobile = [T_time[i] for i in Mobile]
        
        # On passe aux éléments de la catégorie 
        i = 0 
        for s in range(0,len(X_mobile)) :
            i = i+ 1
            V = []
            for k in range(0,len(X_mobile[s])-1):
                t2 = datetime.datetime.fromtimestamp(T_mobile[s][k+1]/ 1e6)
                t1 = datetime.datetime.fromtimestamp(T_mobile[s][k]/ 1e6)
                t = (t2-t1).total_seconds()
                v = np.array([X_mobile[s][k+1]-X_mobile[s][k],Y_mobile[s][k+1]-Y_mobile[s][k]])/t
                V.append(v)
            # On a pas autant de vecteur pour un objet que pour le véhicule ego
            nbr_case_vide = len(V_ego)-len(V)
            Complement = [np.nan]*nbr_case_vide # on va donc compléter par des cases vides pour mettre dans la df
            Vf = V + Complement
            df_vecteurs[catego_name+ str(i)]=Vf
            
        for i in range(1,len(df_vecteurs.columns)):
            nom_col = df_vecteurs.columns[i] # c'est un str 
            liste_vecteur = df_vecteurs[nom_col].values
            newlist = [x for x in liste_vecteur if np.any(np.isnan(x) == False)]
            L =[]
            for s in range(0,len(df_vecteurs)):
                l=[]
                for k in range(0,len(newlist)):
                    l.append(np.dot(df_vecteurs['V_ego'].values[s],newlist[k]))
                L.append(l)
            df_ps[nom_col] = L
            
    return df_ps
    

In [46]:
df_ps = produit_scalaire(scene_name)
df_ps

,human.pedestrian.construction_worker1,vehicle.bus.rigid1,human.pedestrian.adult1,human.pedestrian.adult2,human.pedestrian.adult3,human.pedestrian.adult4,human.pedestrian.adult5,human.pedestrian.adult6,human.pedestrian.adult7,human.pedestrian.adult8,...,human.pedestrian.adult36,human.pedestrian.adult37,human.pedestrian.adult38,human.pedestrian.adult39,vehicle.car1,vehicle.car2,vehicle.car3,vehicle.car4,vehicle.car5,vehicle.truck1
v0 . v_objet,"[-0.803763570787484, -0.7299642206403028, -0.8...","[-88.93043940396025, -80.79211799253255, -85.9...","[11.503929805640832, 10.467694183653496, 11.52...","[-12.983391860986725, -11.774982491830766, -10...","[9.001753834372725, 9.780235590040892, 10.7836...","[10.562749450477828, 11.642111728934822, 11.63...","[0.7629636504585906, 0.3983926989831499, 0.423...","[-13.545170396295749, -13.537986596259735, -10...","[4.929941179916986, 4.3430981842069185, 4.2780...","[0.46704762582764525, 0.6670768568206116, 1.14...",...,"[-2.332240734703208, -2.358092544859399, -2.41...","[3.5866161233017966, 3.582461966074624, 3.9837...","[0.5244381837853189, -0.5742345799907831, -0.5...","[9.344324583925344, 11.717551803542783, 9.3511...","[-87.22466606024574, -85.12526512882924]","[15.415843466665152, 4.022776003263538, 17.813...","[102.73647629773374, 102.98721140585293, 99.19...","[47.35147269381579, 44.00017739413327, 42.7608...","[65.9670867418916, 72.69135516863152, 72.52591...","[29.39826460072205, 34.87938076407666, 40.4955..."
v1 . v_objet,"[-0.7983763432310635, -0.7250185172820243, -0....","[-88.72129830777916, -80.60483813536428, -85.8...","[11.474273069413826, 10.44073945081038, 11.494...","[-12.949335115015394, -11.744140826371593, -10...","[8.973626664703772, 9.753259983412242, 10.7564...","[10.533089929965294, 11.61253817465396, 11.603...","[0.7066969754213832, 0.3429961168302911, 0.362...","[-13.51766292122587, -13.510581806958946, -10....","[4.85750058406033, 4.277540489145838, 4.221733...","[0.4142306819417034, 0.6023547304497217, 1.078...",...,"[-2.3477903063585495, -2.373546463744119, -2.4...","[3.5174438663623033, 3.513369827112971, 3.9068...","[0.46277678957543733, -0.6333779281839012, -0....","[9.292539327594714, 11.652720028689984, 9.2992...","[-86.97840714688735, -84.88354971940957]","[15.366414386334132, 4.010700628235029, 17.783...","[102.4602468664803, 102.7047578143573, 98.9371...","[47.23834075095391, 43.89564435195277, 42.6869...","[65.78745778252299, 72.4934159669941, 72.32842...","[29.312517362569707, 34.778217647835774, 40.39..."
v2 . v_objet,"[-0.8014190365901934, -0.727841209162313, -0.8...","[-88.62547752086373, -80.51474368780059, -85.6...","[11.464786747561378, 10.432073395960385, 11.48...","[-12.939283811174633, -11.734974397519046, -10...","[8.971704043265486, 9.747165063776096, 10.7468...","[10.527094847071755, 11.602446758026717, 11.59...","[0.766761036613572, 0.40343878108604425, 0.429...","[-13.49820919574137, -13.491039929171459, -10....","[4.920199619336895, 4.334721118844078, 4.26878...","[0.471535529602773, 0.6722251831206743, 1.1446...",...,"[-2.3217663149584977, -2.3475335776847217, -2....","[3.581468261061546, 3.5773200662871134, 3.9779...","[0.529754765529189, -0.5651426686171557, -0.58...","[9.315790764457203, 11.681758659816056, 9.3226...","[-86.93039634899573, -84.83824108138171]","[15.364530449035287, 4.009288908553963, 17.751...","[102.38824679035746, 102.63878549700009, 98.85...","[47.18930300020378, 43.84941553887089, 42.6111...","[65.74375493268421, 72.44525832450276, 72.2803...","[29.299407277393502, 34.76202488482389, 40.357..."
v3 . v_objet,"[-0.7998820407226147, -0.7264721517463539, -0....","[-88.26008579064712, -80.18141595074633, -85.3...","[11.418834010945357, 10.390244468259754, 11.43...","[-12.887717172562796, -11.688184374845346, -10...","[8.938230661268003, 9.70898746421744, 10.70348...","[10.486128019970973, 11.555720213596544, 11.55...","[0.7911266541616957, 0.4292959041108837, 0.458...","[-13.440360964903633, -13.433177902829446, -

### Étape 9 : Automatisation du calcul du produit scalaire pour tous les éléments

In [41]:
#imports nécessaires
import matplotlib.pyplot as plt
from math import sqrt
import datetime 
from datetime import timedelta
import pandas as pd
import numpy as np 

def produit_vectoriel(scene_name):
    
    # On crée une dataframe vide 
    df = pd.DataFrame()
    
    scene_token = nusc.field2token('scene','name',scene_name)[0]
    sample_token = nusc.field2token('sample','scene_token',scene_token)# Ensemble des token des échantillons de la scene 
    
    # Pour tracer la trajectoire du véhicule ego
    Liste_ego_poses = get_poses(scene_token) 
    X_coord =[]
    Y_coord =[]
    Z_coord =[]
    T_timestamp = [] # liste des timestamps
    for k in range(0,len(Liste_ego_poses)):
        X_coord.append(Liste_ego_poses[k]['translation'][0])
        Y_coord.append(Liste_ego_poses[k]['translation'][1])
        Z_coord.append(Liste_ego_poses[k]['translation'][2])
        T_timestamp.append(Liste_ego_poses[k]['timestamp'])

    # Ensemble des sample_annotation de la scène :
    sample_annotation_token = []
    for k in sample_token : 
        L = nusc.field2token('sample_annotation','sample_token',k)
        sample_annotation_token = sample_annotation_token + L
    
    # On créé la liste des token des instances présents dans les sample_annotation
    L_instance_token = []
    for k in sample_annotation_token :
        exemple = nusc.get('sample_annotation', k)
        L_instance_token.append(exemple['instance_token'])
    # Puisque pour un même objet, on peut avoir plusieurs annotations, on supprime les doublons de L
    L_instance_token = list(set(L_instance_token))
    
    # Liste des catégories d'objet dans la scène 
    L_category_name = []
    for k in sample_annotation_token :
        exemple = nusc.get('sample_annotation', k)
        L_category_name.append(exemple['category_name'])
    L_category_name = list(set(L_category_name)) #on supprime les doublons 
   
    # On commence par le véhicule ego 
    V_ego = []
    for k in range(0,len(X_coord)-1):
        t2 = datetime.datetime.fromtimestamp(T_timestamp[k+1]/ 1e6)
        t1 = datetime.datetime.fromtimestamp(T_timestamp[k]/ 1e6)
        t = (t2-t1).total_seconds()
        v = np.array([X_coord[k+1]-X_coord[k],Y_coord[k+1]-Y_coord[k],Z_coord[k+1]-Z_coord[k]])/t # Le vecteur a pour norme la vitesse à l'instant k
        V_ego.append(v)
            
    df_vecteurs = pd.DataFrame({'V_ego':V_ego})
    
    # On crée une nouvelle dataframe pour insérer les produits scalaires entre chaque vecteur 
    L =[('v'+str(i)+' . v_objet') for i in range(0,len(V_ego))]
    df_pv = pd.DataFrame(index=L)
    
    # Boucle sur chaque catégorie trouvée
    for j in range(0,len(L_category_name)):   
        catego_name = L_category_name[j]
        catego_token = nusc.field2token('category','name',catego_name)
        category = nusc.get('category',catego_token[0])

        # Ensemble des instances appartenant à la fois à la scène et à la catégorie
        L_instance_category = nusc.field2token('instance','category_token',catego_token[0]) # instance dans la catégorie
        L_instance_category = [x for x in L_instance_category if x in L_instance_token]# et dans la scène 
        L_instance_category
        
        # Il peut y avoir plusieurs éléments de la catégorie dans la scène, ici chaque ligne de L représente 
        # les token des sample_annotation pour un élément de cette catégorie. 

        L=[]
        for k in L_instance_category : 
            instance_category = nusc.get('instance',k)
            nbr_annotations = instance_category['nbr_annotations']
            if nbr_annotations == 1:
                L.append([instance_category['first_annotation_token']])
            else : 
                H = []
                first_token = instance_category['first_annotation_token']
                last_token = instance_category['last_annotation_token']
                current_token = first_token 
        
                s = 0 
                while s!= nbr_annotations :
                    current_ann = nusc.get('sample_annotation', current_token)
                    H.append(current_ann['token'])
                    current_token = current_ann['next'] #On passe au suivant
                    s+=1
                L.append(H)
        
        # Maintenant qu'on a les token de tous les sample_annotation pour tous les éléments d'une même catégorie, 
        # on va pouvoir récupérer leurs positions et tracer leurs trajectoires

        X = []
        Y = []
        Z = []
        T_time = []
        for k in L :
            x = []
            y = []
            z = []
            t=[]
            for j in k :
                sample_annotation = nusc.get('sample_annotation',j)
                x.append(sample_annotation['translation'][0])
                y.append(sample_annotation['translation'][1])
                z.append(sample_annotation['translation'][2])
                t.append(nusc.get('sample',sample_annotation['sample_token'])['timestamp'])
            X.append(x)
            Y.append(y)
            Z.append(z)
            T_time.append(t)
    
        # On crée les listes des objets immobiles et mobiles de la catégorie
        Immo, Mobile = immobile(X,Y)
        
        # On conserve uniquement les objets mobiles 
        X_mobile=[X[i] for i in Mobile]
        Y_mobile=[Y[i] for i in Mobile]
        Z_mobile=[Z[i] for i in Mobile]
        T_mobile = [T_time[i] for i in Mobile]
        
        # On passe aux éléments de la catégorie 
        i = 0 
        for s in range(0,len(X_mobile)) :
            i = i+ 1
            V = []
            for k in range(0,len(X_mobile[s])-1):
                t2 = datetime.datetime.fromtimestamp(T_mobile[s][k+1]/ 1e6)
                t1 = datetime.datetime.fromtimestamp(T_mobile[s][k]/ 1e6)
                t = (t2-t1).total_seconds()
                v = np.array([X_mobile[s][k+1]-X_mobile[s][k],Y_mobile[s][k+1]-Y_mobile[s][k],Z_mobile[s][k+1]-Z_mobile[s][k]])/t
                V.append(v)
            # On a pas autant de vecteur pour un objet que pour le véhicule ego
            nbr_case_vide = len(V_ego)-len(V)
            Complement = [np.nan]*nbr_case_vide # on va donc compléter par des cases vides pour mettre dans la df
            Vf = V + Complement
            df_vecteurs[catego_name+ str(i)]=Vf
            
        for i in range(1,len(df_vecteurs.columns)):
            nom_col = df_vecteurs.columns[i] # c'est un str 
            liste_vecteur = df_vecteurs[nom_col].values
            newlist = [x for x in liste_vecteur if np.any(np.isnan(x) == False)]
            L =[]
            for s in range(0,len(df_vecteurs)):
                l=[]
                for k in range(0,len(newlist)):
                    l.append(np.cross(df_vecteurs['V_ego'].values[s],newlist[k]))
                L.append(l)
            df_pv[nom_col] = L
            
    return df_pv

In [45]:
df_pv = produit_vectoriel(scene_name)
df_pv

,human.pedestrian.construction_worker1,vehicle.bus.rigid1,human.pedestrian.adult1,human.pedestrian.adult2,human.pedestrian.adult3,human.pedestrian.adult4,human.pedestrian.adult5,human.pedestrian.adult6,human.pedestrian.adult7,human.pedestrian.adult8,...,human.pedestrian.adult36,human.pedestrian.adult37,human.pedestrian.adult38,human.pedestrian.adult39,vehicle.car1,vehicle.car2,vehicle.car3,vehicle.car4,vehicle.car5,vehicle.truck1
v0 . v_objet,"[[-0.7949916028358934, 0.29426718753599695, 0....","[[-2.210826714340263, 0.8183404164743777, -1.8...","[[-0.13710553267226447, 0.05074979326972889, -...","[[-0.7198040465293885, 0.26643641466607665, 0....","[[0.0, 0.0, -1.2083037334029028], [0.0, 0.0, -...","[[-0.7941333883883912, 0.29394951832934324, -0...","[[-0.05147409118092342, 0.019053202560566612, ...","[[-0.6513007893511518, 0.2410798439111325, -1....","[[-0.6341130886092232, 0.2347177938724961, -12...","[[-0.8569095792016529, 0.31718620793580554, -1...",...,"[[0.05136207179411222, -0.019011738437186124, ...","[[-0.2916865166918905, 0.1079681478432075, -12...","[[-0.051473575742099834, 0.019053011770208045,...","[[1.0100642276927367, -0.3738766025371668, -5....","[[-0.0, 0.0, 6.43301169042531], [-1.5571242909...","[[-0.42845478960082645, 0.15859310396790277, -...","[[0.0, 0.0, -4.802910235395878], [-0.389281072...","[[0.0, 0.0, 0.6325622922278082], [0.0, 0.0, 0....","[[0.3892810727394075, -0.14409290114183493, -3...","[[0.0, 0.0, -2.719651626405634], [0.0, 0.0, -3..."
v1 . v_objet,"[[-0.7945056287942616, 0.2895830836474052, 0.6...","[[-2.2094752479978728, 0.8053141883070578, -1....","[[-0.13702172080606972, 0.04994196516632921, -...","[[-0.7193640342318661, 0.26219531712322836, 0....","[[0.0, 0.0, -1.2501922799740464], [0.0, 0.0, -...","[[-0.793647938968582, 0.2892704710547037, -0.7...","[[-0.051442625349031254, 0.01874991634999256, ...","[[-0.6509026527220478, 0.23724236871119336, -1...","[[-0.6337254587280725, 0.2309815888942726, -12...","[[-0.8563857550379358, 0.3121372822895575, -10...",...,"[[0.05133067443906177, -0.018709112246722337, ...","[[-0.2915082103111682, 0.10624952598328793, -1...","[[-0.05144211022529262, 0.018749728596608208, ...","[[1.0094467809256407, -0.3679252871273113, -5....","[[-0.0, 0.0, 6.851855363434446], [-1.556172429...","[[-0.4281928775189679, 0.15606864114477875, -2...","[[0.0, 0.0, -5.303052600623097], [-0.389043107...","[[0.0, 0.0, 0.39505587466391034], [0.0, 0.0, 0...","[[0.3890431073375402, -0.14179925051701164, -3...","[[0.0, 0.0, -2.8594647344118194], [0.0, 0.0, -..."
v2 . v_objet,"[[-0.7921024891331536, 0.29372811974802526, 0....","[[-2.202792252426453, 0.8168412994242896, -1.9...","[[-0.13660727146830728, 0.050656824770498626, ...","[[-0.7171881752086132, 0.26594833004511775, 0....","[[0.0, 0.0, -1.1988935222329116], [0.0, 0.0, -...","[[-0.7912473935602551, 0.2934110324792966, -0....","[[-0.051287026938186285, 0.019018299016464044,...","[[-0.6489338687089622, 0.24063820996862628, -1...","[[-0.6318086305409212, 0.23428781456355613, -1...","[[-0.8537954466769204, 0.31660515481561635, -1...",...,"[[0.051175414645920805, -0.01897691085117273, ...","[[-0.29062648598304675, 0.10777036109329295, -...","[[-0.051286513372541434, 0.019018108575614156,...","[[1.006393509171353, -0.37319169833567667, -5....","[[-0.0, 0.0, 6.359854169598513], [-1.551465477...","[[-0.4268977233384602, 0.15830257740780818, -2...","[[0.0, 0.0, -4.726221693922639], [-0.387866369...","[[0.0, 0.0, 0.6581827158166309], [0.0, 0.0, 0....","[[0.3878663693922819, -0.14382893748985132, -3...","[[0.0, 0.0, -2.6931102242826803], [0.0, 0.0, -..."
v3 . v_objet,"[[-0.7881375473207199, 0.2945344509648171, 0.6...","[[-2.1917659733456882, 0.8190836609641232, -2....","[[-0.1359234712152366, 0.050795885951263504, -...","[[-0.713598223879992, 0.2666784012441334, 0.39...","[[0.0, 0.0, -1.171346857567753], [0.0, 0.0, -0...","[[-0.7872867320073043, 0.294216493240228, -0.7...","[[-0.05103030501099581, 0.01907050732461109, -...","[[-0.645685570584